The `CometLoggerCallback` can also be used with other Ray modules, such as `rllib`. Simply pass the callback object to the `tune.run()` method to log the parameters and metrics produced when tuning a model wtih `rllib`.

In [ ]:
%pip install -U "comet_ml>=3.44.0" "ray[rllib]" torch "numpy<2.0"

In [ ]:
import comet_ml

comet_ml.login()

In [ ]:
import os
import ray
from ray import tune, air
from ray.air.integrations.comet import CometLoggerCallback
from ray.rllib.utils.test_utils import check_learning_achieved

config = {
    "env": "CartPole-v0",
    # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
    "num_gpus": int(os.environ.get("RLLIB_NUM_GPUS", "0")),
    "framework": "torch",
    # Run with tracing enabled for tfe/tf2.
    "eager_tracing": False,
    "num_workers": 0,  # Only local worker
}


stop = {"training_iteration": 10}

tuner = tune.Tuner(
    "PPO",
    param_space=config,
    run_config=air.RunConfig(
        stop=stop,
        verbose=2,
        callbacks=[
            CometLoggerCallback(
                project_name="comet-example-rllib-notebook",
                tags=["comet_example"],
            )
        ],
    ),
)

results = tuner.fit()

check_learning_achieved(results, 100)